In [101]:
import json
import pprint
import uuid
import xmltodict

RESULT_TABLE_NAMES = ['prequal_results', 'xml_details', 'products', 'credit_profiles', 'headers', 'risk_models', 'consumer_identities', 'addr_infos', 'employ_infos', 'trade_lines', 'inquiries', 'info_msgs', 'enhanced_pm_data', 'tradeline_amounts']
with open("./data/prequalresult.json", 'r') as f:
    json_file = json.load(f)
json_file

[{'model': 'common.prequalresult',
  'pk': 1,
  'fields': {'loanapp_id': 194,
   'result': True,
   'error_reason': 'no reasons',
   'detail_results': None,
   'fico_v2': '695',
   'report_type': 'CreditProfile',
   'username': 'cho',
   'created_at': '2016-08-04T00:29:03.067Z',
   'xml_data': '<?xml version="1.0" standalone="no"?><NetConnectResponse xmlns="http://www.experian.com/NetConnectResponse"><CompletionCode>0000</CompletionCode><ReferenceId>testing 123</ReferenceId><TransactionId>41588980</TransactionId><Products xmlns="http://www.experian.com/ARFResponse"><CreditProfile><Header><ReportDate>10032017</ReportDate><ReportTime>140607</ReportTime><Preamble>TWA1</Preamble><ARFVersion>07</ARFVersion></Header><RiskModel><ModelIndicator code="F "/><Score>0808</Score><ScoreFactorCodeOne>10</ScoreFactorCodeOne><ScoreFactorCodeTwo>09</ScoreFactorCodeTwo><ScoreFactorCodeThree>05</ScoreFactorCodeThree><ScoreFactorCodeFour>  </ScoreFactorCodeFour><Evaluation code="P"/></RiskModel><RiskModel>

Building PrequalResult table objects

In [102]:
# initialize result object lists
RESULT_OBJ_DICT = dict((key, []) for key in RESULT_TABLE_NAMES)

prequal_results = json_file
for prequal_result in prequal_results:
    prequal_result.update(prequal_result.pop('fields'))
    xml_detail_id = "xd_{}".format(uuid.uuid4())
    prequal_result['xml_detail_id'] = xml_detail_id
    RESULT_OBJ_DICT['prequal_results'].append(prequal_result)
    
    # initializing XML_Detail object
    xml_dict = xmltodict.parse(result['xml_data'], dict_constructor=dict)['NetConnectResponse']
    xml_dict['id'] = xml_detail_id
    RESULT_OBJ_DICT['xml_details'].append(xml_dict)
    product_id = "pd_{}".format(uuid.uuid4())
    xml_dict['product_id'] = product_id
    
    # initializing Product object
    product_obj = xml_dict.pop('Products')
    product_obj['id'] = product_id
    RESULT_OBJ_DICT['products'].append(product_obj)
    credit_profile_id = "cp_{}".format(uuid.uuid4())
    product_obj['credit_profile_id'] = credit_profile_id
    
    # initializing CreditProfile object
    credit_profile_obj = product_obj.pop('CreditProfile')
    credit_profile_obj['id'] = credit_profile_id
    RESULT_OBJ_DICT['credit_profiles'].append(credit_profile_obj)
    
    
    ## handling RiskModels in CreditProfile
    for risk_model in credit_profile_obj.pop('RiskModel'):
        risk_model['id'] = "rm_{}".format(uuid.uuid4())
        risk_model['CreditProfile_id'] = credit_profile_id
        RESULT_OBJ_DICT['risk_models'].append(risk_model)
    
    # initializing Header object in CreditProfile
    header_id = "hd_{}".format(uuid.uuid4())
    credit_profile_obj['header_id'] = header_id
    header_obj = credit_profile_obj.pop('Header')
    header_obj['id'] = header_id
    RESULT_OBJ_DICT['headers'].append(header_obj)
    
    ## handling ConsumerIdentities in CreditProfile
    for consumer_identity in credit_profile_obj.pop('ConsumerIdentity'):
        consumer_identity['id'] = "ci_{}".format(uuid.uuid4())
        consumer_identity['CreditProfile_id'] = credit_profile_id
        RESULT_OBJ_DICT['consumer_identities'].append(consumer_identity)
    
    ## handling AddressInformations in CreditProfile
    for addr_info in credit_profile_obj.pop('AddressInformation'):
        addr_info['id'] = "ai_{}".format(uuid.uuid4())
        addr_info['CreditProfile_id'] = credit_profile_id
        RESULT_OBJ_DICT['addr_infos'].append(addr_info)
       
    ## handling EmploymentInformations in CreditProfile
    for employ_info in credit_profile_obj.pop('EmploymentInformation'):
        employ_info['id'] = "ei_{}".format(uuid.uuid4())
        employ_info['CreditProfile_id'] = credit_profile_id
        RESULT_OBJ_DICT['employ_infos'].append(employ_info)
        
    ## handling TradeLines in CreditProfile
    for trade_line in credit_profile_obj.pop('TradeLine'):
        trade_line['id'] = "tl_{}".format(uuid.uuid4())
        trade_line['CreditProfile_id'] = credit_profile_id
        RESULT_OBJ_DICT['trade_lines'].append(trade_line)
        
        # initializing EnhancedPaymentData object in TradeLine
        enhanced_pm_data_id = "ep_{}".format(uuid.uuid4())
        trade_line['enhanced_pm_data_id'] = enhanced_pm_data_id
        enhanced_pm_data_obj = trade_line.pop('EnhancedPaymentData')
        enhanced_pm_data_obj['id'] = enhanced_pm_data_id
        RESULT_OBJ_DICT['enhanced_pm_data'].append(enhanced_pm_data_obj)
        
        ## handling Amount object in TradeLine
        for amt_obj in trade_line.pop('Amount'):
            amt_obj['id'] = "am_{}".format(uuid.uuid4())
            amt_obj['TradeLine_id'] = trade_line['id']
            RESULT_OBJ_DICT['tradeline_amounts'].append(amt_obj)
    
    # initializing Inquiry object in CreditProfile
    inquiry_id = "iq_{}".format(uuid.uuid4())
    credit_profile_obj['inquiry_id'] = inquiry_id
    inquiry_obj = credit_profile_obj.pop('Inquiry')
    inquiry_obj['id'] = inquiry_id
    RESULT_OBJ_DICT['inquiries'].append(inquiry_obj)
    
    ## handling InformationalMessages in CreditProfile
    for info_msg in credit_profile_obj.pop('InformationalMessage'):
        info_msg['id'] = "im_{}".format(uuid.uuid4())
        info_msg['CreditProfile_id'] = credit_profile_id
        RESULT_OBJ_DICT['info_msgs'].append(info_msg)
    
    
RESULT_OBJ_DICT

{'prequal_results': [{'model': 'common.prequalresult',
   'pk': 1,
   'loanapp_id': 194,
   'result': True,
   'error_reason': 'no reasons',
   'detail_results': None,
   'fico_v2': '695',
   'report_type': 'CreditProfile',
   'username': 'cho',
   'created_at': '2016-08-04T00:29:03.067Z',
   'xml_data': '<?xml version="1.0" standalone="no"?><NetConnectResponse xmlns="http://www.experian.com/NetConnectResponse"><CompletionCode>0000</CompletionCode><ReferenceId>testing 123</ReferenceId><TransactionId>41588980</TransactionId><Products xmlns="http://www.experian.com/ARFResponse"><CreditProfile><Header><ReportDate>10032017</ReportDate><ReportTime>140607</ReportTime><Preamble>TWA1</Preamble><ARFVersion>07</ARFVersion></Header><RiskModel><ModelIndicator code="F "/><Score>0808</Score><ScoreFactorCodeOne>10</ScoreFactorCodeOne><ScoreFactorCodeTwo>09</ScoreFactorCodeTwo><ScoreFactorCodeThree>05</ScoreFactorCodeThree><ScoreFactorCodeFour>  </ScoreFactorCodeFour><Evaluation code="P"/></RiskModel>

Parse XML data string and build XML_Detail table objects

In [104]:
RESULT_OBJ_DICT['tradeline_amounts']

[{'Qualifier': {'@code': 'L'},
  'Value': '00000350',
  'id': 'am_124d16d8-dfbd-4087-8524-166dc13ef679',
  'TradeLine_id': 'tl_04682736-ca7d-49b6-9199-0d4ac3f03b2d'},
 {'Qualifier': {'@code': 'H'},
  'Value': '00000056',
  'id': 'am_637c590d-e17a-4279-a161-626e0dff2ebe',
  'TradeLine_id': 'tl_04682736-ca7d-49b6-9199-0d4ac3f03b2d'},
 {'Qualifier': {'@code': 'L'},
  'Value': '00011500',
  'id': 'am_f6a4bd9c-0ceb-4726-ac71-481c211371a0',
  'TradeLine_id': 'tl_fbddedcd-55fc-44ad-b55e-a2605f221311'},
 {'Qualifier': {'@code': 'H'},
  'Value': '00011440',
  'id': 'am_8e9b6b5e-429e-4f8a-ab85-4c01d5a62fe4',
  'TradeLine_id': 'tl_fbddedcd-55fc-44ad-b55e-a2605f221311'},
 {'Qualifier': {'@code': 'L'},
  'Value': '00012900',
  'id': 'am_2864c251-c44b-46c7-828b-9340df49f018',
  'TradeLine_id': 'tl_bed432f2-c6ca-4dc7-98ef-5e9fbb2be900'},
 {'Qualifier': {'@code': 'H'},
  'Value': '00008979',
  'id': 'am_00cbf555-e8e0-4ff2-b913-050954fb0f3f',
  'TradeLine_id': 'tl_bed432f2-c6ca-4dc7-98ef-5e9fbb2be900'}